In [1]:
# import libraries

from openai import OpenAI
import os
from dotenv import load_dotenv


In [6]:
from pathlib import Path

import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o-mini") 

extracted_file = Path(r'C:\Users\herms\Desktop\Civic_Tech_Research_Project\PreliminarWork\preprocessing\extracted_text_no_headers_footers_clean_vers_001.txt')

text_content = []

with open("extracted_text_no_headers_footers_clean_vers_001.txt", "r", encoding="utf-8") as text_file:
    text_full = text_file.read()

    
    # Count the number of tokens    
    instruction_tokens = encoding.encode(text_full)
    num_tokens = len(instruction_tokens)
    print(f"Number of tokens : {num_tokens}")

    text_content.append(text_full) #save for future processing
     
    print("\n ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")  
        

Number of tokens : 16787

 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [15]:
## if needed, for a long document, it is going to be necessary to break the document in pieces and make a recursive summarization 

In [8]:
#token encoding for the instruction

# Load the encoding for the specific model you're using
encoding = tiktoken.encoding_for_model("gpt-4o-mini")    

# Instruction text
instruction = f"Has un resumen de una minuta de reunión de un gobierno local.\
La intención de este ejercicio es proveer al ciudadano común con información relevante sobre el desempeño de su gobierno local. \
Instrucciones especificas: \
- Enfatiza los problemas, peticiones y soluciones o acuerdos que se llegan \
- Indica los nombres de las personas que participan cuando sea necesario.\
- Indica datos concretos \
- No emitas juicios de valor, manten neutralidad\
- Identifica fechas\
- Identifica los temas que se trataron."

# Encode the text
instruction_tokens = encoding.encode(instruction)

# Count the number of tokens
num_tokens = len(instruction_tokens)

print(f"Number of tokens: {num_tokens}")


Number of tokens: 101


In [9]:
# loading API key
load_dotenv()
key_ct = os.environ["ct_api_key"]

In [12]:

client = OpenAI(api_key = key_ct)

response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
        {"role": "system", "content" : instruction},
        {"role":"user", "content":text_content[0]},
    ],
    temperature = 0.7,
    max_tokens = 5000,
)

response_summary_TOTAL_vers3 = response.choices[0].message.content
print("Summary:", response_summary_TOTAL_vers3)   

Summary: **Minuta de Reunión - Concejo Metropolitano de Quito**

**Fecha:** 04 de febrero de 2019  
**Hora de inicio:** 11:35 AM  
**Hora de finalización:** 12:43 PM  
**Asistentes:** 18 concejales y el Alcalde Metropolitano, Dr. Mauricio Rodas Espinel.

### Temas Tratados:

1. **Segundo Debate del Proyecto de Ordenanza Metropolitana Reformatoria a la Ordenanza No. 232.**
   - Se busca regular la prestación del servicio de taxi en el Distrito Metropolitano de Quito.

### Problemas Identificados:
- **Cupos insuficientes:** El estudio previo determinó que hay 8,693 cupos, mientras que 12,905 aplicantes fueron calificados como idóneos.
- **Crecimiento poblacional y demanda:** Se ha incrementado la necesidad del servicio de taxi debido a la migración y el crecimiento demográfico.

### Peticiones Realizadas:
- **Claridad Jurídica:** Se solicitó al Procurador Metropolitano, Dr. Gianni Frixone, que aclare si la decisión del Concejo está enmarcada en la legalidad y si existen riesgos jurídicos

In [13]:


with open("response_summary_TOTAL_vers4.txt", "w", encoding="utf-8") as text_file:
 
    text_file.write(response_summary_TOTAL_vers3)  # Write the block text to the file, with spacing between blocks

print("Text extraction completed and saved to 'response_summary_TOTAL_vers4'.")


Text extraction completed and saved to 'response_summary_TOTAL_vers4'.
